# TP2
1. Um  programa imperativo pode ser descrito por um modelo do tipo Control Flow Automaton (CFA) como ilustrado no exemplo seguinte

<img src="automato.png" alt="autómato" style="width: 600px;"/>

Este programa implementa a multiplicação de dois inteiros a,b fornecidos como “input”, e com precisão limitada a $\,n\,$ bits (fornecido como parâmetro do programa). 

Note-se que:

    - Existe a possibilidade de alguma das operações do programa produzir um erro de “overflow”. 
    - Os nós do grafo representam ações  que actuam sobre os “inputs” do nó e produzem um output com as operações indicadas
    - Os ramos do grafo representam ligações que transferem o “output” de um nodo para o “input” do nodo seguinte. Esta transferência é condicionada pela satisfação da condição associada ao ramo


a. Construa um FOTS usando BitVector de tamanho $n$ que descreva o comportamento deste autómato. Para isso identifique as variáveis do modelo, o estado inicial e a relação de transição.

    
b. Verifique se $$\;\;\mathsf{P}\,\equiv\,(x*y + z = a*b)\;\;$$é um invariante deste comportamento.

In [13]:
from pysmt.shortcuts import *
from z3 import *

In [14]:
def declare(i, n):
    state = {}
    state['pc'] = Int('pc'+str(i))
    state['x'] = BitVec('x'+str(i), n)
    state['y'] = BitVec('y'+str(i), n)
    state['z'] = BitVec('z'+str(i), n)


    return state


In [18]:
def init(state, a, b):
    return And(state['pc']== 0, state['x'] == a, state['y'] == b, state['z'] == 0)

In [19]:
def trans(curr, prox, n):
    same_values = And(
        prox['x'] == curr['x'],
        prox['y'] == curr['y'],
        prox['z'] == curr['z']    
    )
    
    t0 = And(
        curr['pc'] == 0,
        prox['pc'] == 1,
        same_values
    )
    
    # y = 0
    t1 = And(
        curr['y'] == 0,
        curr['pc'] == 1,
        prox['pc'] == 5,
        same_values
    )
    
    # caso de paragem
    stop = And(
        curr['pc'] == 5,
        prox['pc'] == 5,
        prox['pc'] == curr['pc'],
        same_values
    )
    
    # y != 0 ^ odd(y)
    t2 = And(
        curr['y'] != 0,
        (curr['y'] % 2) == 1,
        curr['pc'] == 1,
        prox['pc'] == 2,
        same_values
    )
    
    #Q2 -> Q1
    t5 = And(
        curr['pc'] == 2,
        prox['pc'] == 1,
        prox['x'] == curr['x'],
        prox['y'] == curr['y']-1,
        prox['z'] == curr['z'] + curr['x']
    )
    
    # y != 0 ^ even(y)
    t3 = And(
        curr['y'] != 0,
        (curr['y'] % 2) == 0,
        curr['pc'] == 1,
        prox['pc'] == 3,
        same_values
    )
    
    # Q3 -> Q1
    t4 = And(
        curr['pc'] == 3,
        prox['pc'] == 1,
        
        prox['x'] == curr['x'] << 1,
        prox['y'] == curr['y'] >> 1,
        prox['z'] == curr['z'],
     
        curr['x'] > (curr['x'] << 1),
        prox['x'] > curr['x']
    )
    
    # caso de overflow
    overflow = And(
        curr['pc'] == 3,
        prox['pc'] == 4,

        prox['x'] == curr['x'] << 1,
        prox['y'] == curr['y'] >> 1,
        prox['z'] == curr['z'],
    
        curr['x'] > (curr['x'] << 1),
        prox['x'] < curr['x']
    )


    
    magia = And(
        prox['x'] == curr['x'] << BitVecVal(1, n),
        prox['y'] == curr['y'] >> BitVecVal(1, n),
        prox['z'] == curr['z'],
        
        curr['pc'] == 3,
        
        Or(
            And(ULT(curr['x'], BitVecVal(2**(n-1), n)), prox['pc'] == 1),
            And(UGE(curr['x'], BitVecVal(2**(n-1), n)), prox['pc'] == 4),
        )
    )
    
    # caso de paragem no overflow
    error = And(
        curr['pc'] == 4,
        prox['pc'] == 4,
        prox['pc'] == curr['pc'],
        same_values
    )
    
    return Or(t0, t1, t2, t3, magia, t5, stop, error)

In [21]:
#está a dar overflow quando n deve dar, dar fix a isto

def gera_traco(declare,init,trans,k, n):

    s = Solver()
    
    
    trace = [declare(i, n) for i in range(k)]

    # adicionar o estado inicial
    s.add(init(trace[0],150,2))
    
    # adicionar as transições
    for i in range(k - 1):
        s.add(trans(trace[i], trace[i+1], n))
    
    
    check = s.check()
    if check == sat:
        m = s.model()
        for i in range(k):
            print("Passo ", i)
            for v in trace[i]:
                print(v, "=", m[trace[i][v]])
            print("----------------")
    else:
        print(check)
                
gera_traco(declare,init,trans,20, 16)

Passo  0
pc = 0
x = 150
y = 2
z = 0
----------------
Passo  1
pc = 1
x = 150
y = 2
z = 0
----------------
Passo  2
pc = 3
x = 150
y = 2
z = 0
----------------
Passo  3
pc = 1
x = 300
y = 1
z = 0
----------------
Passo  4
pc = 2
x = 300
y = 1
z = 0
----------------
Passo  5
pc = 1
x = 300
y = 0
z = 300
----------------
Passo  6
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  7
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  8
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  9
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  10
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  11
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  12
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  13
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  14
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  15
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  16
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  17
pc = 5
x = 300
y = 0
z = 300
----------------
Passo  18
pc